In [1]:
!pip install pysqlite3
!pip install pysqlite3-binary
!pip install pypdf
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysqlite3: filename=pysqlite3-0.5.4-cp311-cp311-linux_x86_64.whl size=159236 sha256=fa7f64f719f7829dae33fb355025b828dba5b48f79982681adef9da6d2f230d4
  Stored in directory: /root/.cache/pip/wheels/83/05/4e/8fb9d7378ff72e4fd02b69df7a6052c34d33cea2a0231ca232
Successfully built pysqlite3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import pysqlite3
import sys

In [3]:
# Load and split the pdf
pdf_loader = PyPDFLoader("/content/the_nestle_hr_policy_pdf_2012.pdf")
pdf_pages = pdf_loader.load_and_split()
print(len(pdf_pages))

8


In [4]:
#Split the PDF pages into smaller chunks and print the number of chunks.
doc_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
docs = doc_splitter.split_documents(pdf_pages)
doc_list = [doc.page_content for doc in docs]
print(doc_list[1])

Policy
Mandatory
September  20
12
Issuing departement
Hum
an Resources
Target audience 
All
 employees
Approver
Executive Board, Nestlé S.A.
Repository
All Nestlé Principles and Policies, Standards and  
Guidelines can be found in the Centre online repository at:  
http://intranet.nestle.com/nestledocs
Copyright and confidentiality
Al
l rights belong to Nestec Ltd., Vevey, Switzerland.
© 2012, Nestec Ltd.
Design
Nestec Ltd., Corporate Identity & Design,  
Vevey, Switzerland
Production
brain’print GmbH, Switzerland
Paper
This report is printed on BVS, a paper produced  
from well-managed forests and other controlled sources  
certified by the Forest Stewardship Council (FSC).


**Create vector representations for text chunks using Chroma dB and OpenAI's embeddings.**

In [5]:
!pip install chromadb openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:

In [6]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [7]:
import chromadb
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [12]:
# Initilize the OpenAI Embedding model
embedding_model = OpenAIEmbeddings()

In [13]:
# Create a Chroma vector store from text chunks
vector_store = Chroma.from_texts(texts=doc_list, embedding=embedding_model, persist_directory="./chroma_db")

In [14]:
vector_store.persist()

<ipython-input-14-261e6ea3744e>:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [15]:
## Create my own query
query = "What is working conditions?"

#Get the similar embedding from the vector_store
results = vector_store.similarity_search(query, k=3)

In [16]:
for doc in results:
    print(doc.page_content)
    print("\n............")

of mutual trust. HR ensures that a respectful 
dialogue is present and the voice of the 
employees is heard.
Corporate policy: 
Policy on Conditions of Work and Employment
 Employment and working conditions

............
and Wellness Company.
As such, we provide flexible working 
conditions whenever possible and encourage our 
employees to have outside interests especially 
community involvement. 
Those with line management responsibilities 
are required to take personal ownership of safety 
and health within their area of responsibility and 
are encouraged to develop their capability in this 
area.
Nestlé’s commitment however goes beyond 
its own employees. We care about all people 
working inside or outside our premises under 
contractual obligations with service providers 
and we insist that they also take steps so that 
adequate working conditions are made available 
to them.
We believe that it is essential to build a 
relationship based on trust and respect of 
employees at all le

In [17]:
#Define the chat-gpt model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Retrieve similar embedding from chroma db i.e vector_store
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Define RetrievalQA which takes user query and find the similar content from pdfs and send to chat-gpt model to get the
# answer
qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True
        )

<ipython-input-17-01102e2510f0>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [18]:
response = qa_chain({"query": query})

<ipython-input-18-7e13e1e84d4b>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"query": query})


In [19]:
response['result']

"Working conditions refer to the environment in which employees carry out their daily tasks. This includes factors such as the physical work environment, hours of work, safety regulations, benefits, and overall atmosphere in the workplace. In the context provided, working conditions are emphasized as an important aspect of Nestlé's corporate policy, ensuring that employees have a safe, respectful, and supportive environment to work in."

**Implement the Gradio to show the demo**

In [21]:
!pip install gradio
!pip install --upgrade gradio

In [22]:
import gradio as gr

In [23]:
def greet(query):
    response = qa_chain({"query": query})
    return response['result']


chat_demo = gr.Interface(
    fn=greet,
    inputs=['text'],
    outputs=['text']
)

In [24]:
chat_demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ef628e6efceb812379.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
